# Projeto 1 - Ciência dos Dados

Nome: Arthur Martins de Souza Barreto

Nome: Giselle Vieira de Melo

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# bibliotecas adicionais
import re 

In [3]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\arthu\Desktop\Documentos\Insper\2º_Periodo\Ciencia_dos_dados\twitter


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [4]:
filename = 'La casa de papel.xlsx'

In [5]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevante
0,coisas q serão proibidas quando eu for preside...,1
1,agr que tô terminando de assistir la casa de p...,1
2,paguem minha terapia (la casa de papel vc me p...,0
3,to com pena de terminar la casa de papel,0
4,"olha, quem me segue aqui sabe o tanto que odei...",1


In [6]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste,Relevante
0,vou assistir a última temperada de la casa de ...,1
1,só quero chegar em casa e me entupir de brigad...,1
2,"toda vez, a gnt depois que acaba de ver la cas...",0
3,@esteseverino na verdade eles se comunicavam p...,1
4,não tô chorando horrores com o final de la cas...,0


___
## Classificador automático de sentimento


Tivemos varios tweets, é muitos deles eram vagos, não dava para discernir se a pessoa gostava ou não da série La casa de papel. Deste modo, foi considerado relevando todos os tweets que foi possível discernir se a pessoa gostava ou não da série.

In [7]:
# FUNÇÕES AUXILIARES DO SISTEMA para a montagem da base de dados do treinamento
def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;\/)|,''""@#(*]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed.lower()

def frase_para_palavras(df):
    '''
    função para percorrer linha a linha uma coluna do dataframe e concatenar as palavras em uma unica lista
    para ser usada no value_counts()
    '''
    data = ''
    
    for tweet in df:
        # o tweet é a frase que a pessoa postou, que se encontra em cada linha da nossa coluna
        data += tweet
        
    # dando o split agora, para separar por virgula
    data = data.split()
    
    return data

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [8]:
#convertendo em variáveis categóricas:
train['Treinamento'] = train['Treinamento'].astype('category')
test['Teste'] = test['Teste'].astype('category')

In [9]:
#limpando os caracteres indesejados do dataframe
train['Clean']=train['Treinamento'].apply(cleanup)
test['Clean']=test['Teste'].apply(cleanup)

In [10]:
test

,Teste,Relevante,Clean
0,vou assistir a última temperada de la casa de ...,1,vou assistir a última temperada de la casa de ...
1,só quero chegar em casa e me entupir de brigad...,1,só quero chegar em casa e me entupir de brigad...
2,"toda vez, a gnt depois que acaba de ver la cas...",0,toda vez a gnt depois que acaba de ver la casa...
3,@esteseverino na verdade eles se comunicavam p...,1,esteseverino na verdade eles se comunicavam pe...
4,não tô chorando horrores com o final de la cas...,0,não tô chorando horrores com o final de la cas...
...,...,...,...
195,acabei la casa de papel,0,acabei la casa de papel
196,precisamos conversar sobre o final dessa tempo...,0,precisamos conversar sobre o final dessa tempo...
197,"eu odeio lá casa de papel, mataram a tóquio va...",1,eu odeio lá casa de papel mataram a tóquio vai...
198,preguiça de ver essa parte 5 de la casa de papel,0,preguiça de ver essa parte 5 de la casa de papel


In [11]:
# post relevante
post_rel = train.loc[(train['Relevante'] == 1)]
# post irrelevante
post_irrel = train.loc[(train['Relevante'] == 0)]

In [50]:
# agora precisamos quebrar o texto em palavras, usando a função aux que converter frase em palavras
# da coluna Clean, cuja post é relevante, temos:
data_rel   = frase_para_palavras(post_rel['Clean'])
data_irrel = frase_para_palavras(post_irrel['Clean'])

# criando a serie dos posts relevantes e irrelevantes
serie_rel = pd.Series(data_rel)
serie_irrel = pd.Series(data_irrel)
# Formando uma base de daddos com todas as palavras nele:
data_total = data_rel + data_irrel

#criando a serie com a base total de dados
serie_total = pd.Series(data_total)

print(len(serie_rel))
print(len(serie_irrel))
print(len(serie_total))

2275
2111
4386


#### Frequências absolutas

Palavras em um texto são variáveis **qualitativas nominais**, portanto usaremos `value_counts()` para obter a tabela de frequências relativas e absolutas:

In [46]:
# construindo a tabela de relevante e irrelevante
tabela_rel   = serie_rel.value_counts()
tabela_irrel = serie_irrel.value_counts()
tabela_total = serie_total.value_counts()

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [52]:
# postagens relevantes
post_rel.head()

,Treinamento,Relevante,Clean
0,coisas q serão proibidas quando eu for preside...,1,coisas q serão proibidas quando eu for preside...
1,agr que tô terminando de assistir la casa de p...,1,agr que tô terminando de assistir la casa de p...
4,"olha, quem me segue aqui sabe o tanto que odei...",1,olha quem me segue aqui sabe o tanto que odeio...
5,simplesmente devastada com esse final da 1° pa...,1,simplesmente devastada com esse final da 1° pa...
6,que final fdp do lá casa de papel,1,que final fdp do lá casa de papel


In [53]:
# postagens irrelevantes
post_irrel.head()

,Treinamento,Relevante,Clean
2,paguem minha terapia (la casa de papel vc me p...,0,paguem minha terapia la casa de papel vc me paga
3,to com pena de terminar la casa de papel,0,to com pena de terminar la casa de papel
7,ai la casa de papel me mata pqp,0,ai la casa de papel me mata pqp
10,criadores de lá casa de papel deixaram mto na ...,0,criadores de lá casa de papel deixaram mto na ...
11,não tenho paciência para essas cenas aleatória...,0,não tenho paciência para essas cenas aleatória...


### Calculando as probabilidades:

In [64]:
p_rel = len(serie_rel)/len(serie_total)
print('A probabilidade das postagens serem relevantes é: {0:.3f}'.format(p_rel))

A probabilidade das postagens serem relevantes é: 0.519


In [65]:
p_irrel = len(serie_irrel)/len(serie_total)
print('A probabilidade das postagens serem irrelevantes é: {0:.3f}'.format(p_irrel))

A probabilidade das postagens serem irrelevantes é: 0.481


### Suavização de laplace

___
### Concluindo

___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CORRIGIU separação de espaços entre palavras e emojis ou entre emojis e emojis
* CRIOU categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante. Pelo menos quatro categorias, com adição de mais tweets na base, conforme enunciado. (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item 6. Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**